# 03. Notebook: Sentiment RNN

영화 리뷰들을 분석하여 해당 리뷰가 긍정적 리뷰인지 / 부정적 리뷰인지 예측하는 모델만들기

---

In [1]:
import os
os.getcwd()

'C:\\Users\\USER\\Desktop\\Sentiment_Analysis'

## 01. Data pre-processing

### 1-0.Load in and visualize the data

In [7]:
import numpy as np

# Read data from text files
with open('data/reviews.txt', 'r') as f:
    reviews = f.read()
    
with open('data/labels.txt', 'r') as f:
    labels = f.read()

In [15]:
print(f"The number of Review's characters: {len(reviews)}")
print()
print(f"Review data sample: \n {reviews[:2000]}")
print()
print(f"Label data sample: \n {labels[:100]}")

The number of Review's characters: 33678267

Review data sample: 
 bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terr

**Comment** :



대략 3천3백만개의 글자로 이루어진 리뷰 텍스트 데이터가 있고


각 리뷰에 대한 긍정/부정 정답, 즉 라벨데이터도 단순 텍스트덩어리로 되어있구나


대충 리뷰를 읽어보니 Airport77이라는 영화의 리뷰인거같고 각 리뷰간의 구분은 `\n`, 즉 줄바꿈으로 되는거같다

### 1-1. Data pre-processing

텍스트데이터 -> 단어

단어 -> 정수 (with Embedding layer)가 되어야 RNN에 사용할 수 있음

In [17]:
# 텍스트 정제
# 소문자로 변경, 구두점 삭제
from string import punctuation
print(punctuation)

reviews = reviews.lower()
all_text = ''.join([c for c in reviews if c not in punctuation])

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [19]:
# 리뷰간 구분하여 리스트만들기, 즉 [리뷰1, 리뷰2, 리뷰3...]
reviews_split = all_text.split('\n')

# 딕셔너리 만들기 위해!
# 리스트로 쪼개진 리뷰들을 띄어쓰기로 구분하여 다시 덩어리로 만들기
all_text = ' '.join(reviews_split)

In [22]:
# 단어 리스트로 만들기 (리뷰간 구분없는)
words = all_text.split()
words[:10]

['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', 'it', 'ran', 'at', 'the']

### 1-2. Encoding the words

Embedding Lookup Table을 사용하려면 입력이 '정수값'이어야 하므로 단어<-> 정수 Dictionary를 만들자 (빈도수순으로 정렬해서~)

In [24]:
from collections import Counter
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)

<span style="color:red">[!]</span> 추후에 패딩을 '0'으로 할것이므로 겹치지 않게 정수값 시작을 '1'부터로 하자

In [26]:
vocab_to_int = { word: ii for ii, word in enumerate(vocab, start=1)}

In [27]:
vocab_to_int

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'br': 7,
 'it': 8,
 'in': 9,
 'i': 10,
 'this': 11,
 'that': 12,
 's': 13,
 'was': 14,
 'as': 15,
 'for': 16,
 'with': 17,
 'movie': 18,
 'but': 19,
 'film': 20,
 'you': 21,
 'on': 22,
 't': 23,
 'not': 24,
 'he': 25,
 'are': 26,
 'his': 27,
 'have': 28,
 'be': 29,
 'one': 30,
 'all': 31,
 'at': 32,
 'they': 33,
 'by': 34,
 'an': 35,
 'who': 36,
 'so': 37,
 'from': 38,
 'like': 39,
 'there': 40,
 'her': 41,
 'or': 42,
 'just': 43,
 'about': 44,
 'out': 45,
 'if': 46,
 'has': 47,
 'what': 48,
 'some': 49,
 'good': 50,
 'can': 51,
 'more': 52,
 'she': 53,
 'when': 54,
 'very': 55,
 'up': 56,
 'time': 57,
 'no': 58,
 'even': 59,
 'my': 60,
 'would': 61,
 'which': 62,
 'story': 63,
 'only': 64,
 'really': 65,
 'see': 66,
 'their': 67,
 'had': 68,
 'we': 69,
 'were': 70,
 'me': 71,
 'well': 72,
 'than': 73,
 'much': 74,
 'get': 75,
 'bad': 76,
 'been': 77,
 'people': 78,
 'will': 79,
 'do': 80,
 'other': 81,
 'also': 82,
 'into':

리뷰 단어 -> 정수로 변환

아까 위에서 만들어놨던 (정제된) 리뷰 리스트 `reviews_split` [리뷰1, 리뷰2, 리뷰3...]을 이제 정수로 변환해주자

* [리뷰1, 리뷰2, ...] 리스트를 for loop로 돌린다
* 하나씩 나오는 리뷰를 `split()`을 통해 단어가 나오게끔 for loop로 돌린다
* 튀어나오는 단어들을 딕셔너리로 변환하여 리스트에 쌓는다
* 결론적으로 [[리뷰1단어1, 리뷰1단어2,...], [리뷰2단어1, 리뷰2단어2], ....]와 같은 이중 리스트가 됨

In [30]:
reviews_ints = []

for review in reviews_split:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])

### 1-3. Encoding the labels

아래처럼 각 리뷰에 대한 긍정/부정 라벨을 쪼개고, 

Positive는 1, Negative는 0으로 바꿔주자

In [32]:
labels[:50]

'positive\nnegative\npositive\nnegative\npositive\nnegat'

In [34]:
labels_split = labels.split('\n')
encoded_labels = [1 if label == 'positive' else 0 for label in labels_split]
print(encoded_labels[:10])

[1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
